<a href="https://colab.research.google.com/github/Chandralekha005/86_VasaviKalyaniSure/blob/main/MD%26A_GENERATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("securities-exchange-commission/financial-statement-extracts")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'financial-statement-extracts' dataset.
Path to dataset files: /kaggle/input/financial-statement-extracts


In [ ]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/securities-exchange-commission/financial-statement-extracts/versions/2"

for root, dirs, files in os.walk(dataset_path):
    print("\nDirectory:", root)
    for f in files:
        print("  File:", f)



Directory: /root/.cache/kagglehub/datasets/securities-exchange-commission/financial-statement-extracts/versions/2
  File: 2017q1.json
  File: manual.pdf
  File: 2015q3.json
  File: 2017q2.json
  File: 2015q4.json
  File: 2016q3.json
  File: 2016q4.json
  File: 2016q1.json
  File: 2016q2.json


In [ ]:
import json

file_path = "/root/.cache/kagglehub/datasets/securities-exchange-commission/financial-statement-extracts/versions/2/2017q1.json"

with open(file_path, "r") as f:
    data = json.load(f)

print("Type of data:", type(data))

Type of data: <class 'dict'>


In [ ]:
print("Number of records:", len(data))

Number of records: 5


In [ ]:
from pprint import pprint

# Print the keys of the dictionary to understand its structure
print("Available keys in the data dictionary:")
pprint(data.keys())

Available keys in the data dictionary:
dict_keys(['num.txt', 'pre.txt', 'readme.htm', 'sub.txt', 'tag.txt'])


In [ ]:
from pprint import pprint

# Accessing 'pre.txt' content, as 0 is not a valid key for this dictionary.
# The content inside these files are large, so printing only the first 1000 characters.
print(data["num.txt"][:3000])

adsh	tag	version	coreg	ddate	qtrs	uom	value	footnote
0001654954-17-000061	AccountsPayableCurrent	us-gaap/2015		20141231	0	USD	726627.0000	
0001654954-17-000061	AccountsPayableCurrent	us-gaap/2015		20151231	0	USD	1423503.0000	
0001654954-17-000061	AccountsPayableCurrent	us-gaap/2015		20160930	0	USD	1851778.0000	
0000940944-17-000002	AccountsPayableCurrent	us-gaap/2015		20160531	0	USD	241900000.0000	
0000940944-17-000002	AccountsPayableCurrent	us-gaap/2015		20161130	0	USD	249600000.0000	
0001635881-17-000007	AccountsPayableCurrent	us-gaap/2015		20141231	0	USD	161826000.0000	
0001635881-17-000007	AccountsPayableCurrent	us-gaap/2015		20151231	0	USD	94566000.0000	
0001098009-17-000004	AccountsPayableCurrent	us-gaap/2015		20160630	0	USD	4020.0000	
0001098009-17-000004	AccountsPayableCurrent	us-gaap/2015		20160930	0	USD	7320.0000	
0000899866-17-000002	AccountsPayableCurrent	us-gaap/2015		20151231	0	USD	57360000.0000	
0000899866-17-000002	AccountsPayableCurrent	us-gaap/2015		20160930	0	USD	448

📂 Dataset Files → Project Components
🔹 sub.txt → “Company & Filing Context”

Used for:

Company name

Filing type (10-Q / 10-K)

Period (quarter/year)

📌 Appears in:

Financial Performance Overview

Report metadata

Citations

🔹 num.txt → “Financial Facts Engine” ⭐

Used for:

Revenue

Expenses

Net Income

Cash flow

📌 Drives:

KPI computation

YoY / QoQ changes

Trend analysis

Risk classification

Predictive outlook

This is your core table.

🔹 tag.txt → “Financial Language Translator”

Used for:

Explaining what “NetIncomeLoss” means

Human-readable descriptions in MD&A

📌 Prevents:

Confusing technical terms in output

🔹 pre.txt → “Statement Classification”

Used for:

Identifying Balance Sheet vs Income Statement vs Cash Flow

📌 Helps:

Structure MD&A sections correctly

In [ ]:
from io import StringIO
import pandas as pd

num_df = pd.read_csv(StringIO(data["num.txt"]), sep="\t")
sub_df = pd.read_csv(StringIO(data["sub.txt"]), sep="\t")
tag_df = pd.read_csv(StringIO(data["tag.txt"]), sep="\t")
pre_df = pd.read_csv(StringIO(data["pre.txt"]), sep="\t")

print(num_df.shape)
print(sub_df.shape)
print(tag_df.shape)
print(pre_df.shape)

(3313595, 9)
(6272, 36)
(81580, 9)
(1009876, 10)


In [ ]:
print(num_df.columns)
print(sub_df.columns)
print(pre_df.columns)
print(tag_df.columns)

Index(['adsh', 'tag', 'version', 'coreg', 'ddate', 'qtrs', 'uom', 'value',
       'footnote'],
      dtype='object')
Index(['adsh', 'cik', 'name', 'sic', 'countryba', 'stprba', 'cityba', 'zipba',
       'bas1', 'bas2', 'baph', 'countryma', 'stprma', 'cityma', 'zipma',
       'mas1', 'mas2', 'countryinc', 'stprinc', 'ein', 'former', 'changed',
       'afs', 'wksi', 'fye', 'form', 'period', 'fy', 'fp', 'filed', 'accepted',
       'prevrpt', 'detail', 'instance', 'nciks', 'aciks'],
      dtype='object')
Index(['adsh', 'report', 'line', 'stmt', 'inpth', 'rfile', 'tag', 'version',
       'plabel', 'negating'],
      dtype='object')
Index(['tag', 'version', 'custom', 'abstract', 'datatype', 'iord', 'crdr',
       'tlabel', 'doc'],
      dtype='object')


🟢 STEP 2: Merge the Core Tables (num + sub)
🎯 Goal of Step 2

Combine:

WHAT the number is (num_df)

WHO & WHEN reported it (sub_df)

So that each financial value has company + period context.

In [ ]:
merged_df = num_df.merge(
    sub_df,
    on="adsh",
    how="left"
)

print("Before merge (num):", num_df.shape)
print("After merge:", merged_df.shape)

Before merge (num): (3313595, 9)
After merge: (3313595, 44)


In [ ]:
merged_df.columns

Index(['adsh', 'tag', 'version', 'coreg', 'ddate', 'qtrs', 'uom', 'value',
       'footnote', 'cik', 'name', 'sic', 'countryba', 'stprba', 'cityba',
       'zipba', 'bas1', 'bas2', 'baph', 'countryma', 'stprma', 'cityma',
       'zipma', 'mas1', 'mas2', 'countryinc', 'stprinc', 'ein', 'former',
       'changed', 'afs', 'wksi', 'fye', 'form', 'period', 'fy', 'fp', 'filed',
       'accepted', 'prevrpt', 'detail', 'instance', 'nciks', 'aciks'],
      dtype='object')

✅ 1️⃣ What this merged CSV actually is (Big Picture)

After merging, you now have one master table that represents:

📄 SEC financial facts + company metadata per filing

Each row typically represents:

One financial fact (like Revenue, Assets, NetIncome)

For one company

For one reporting period

Reported in one SEC filing

This table is the raw foundation for:

Financial analysis

Trend detection

Risk assessment

MD&A generation

👉 This is EXACTLY what MD&A is written from in real companies.

FULL WORKING CODE

In [ ]:
import json
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from tqdm import tqdm
import openai

In [ ]:
def load_financials(path):
    with open(path, "r") as f:
        data = json.load(f)
    df = pd.DataFrame(data["periods"])
    df = df.sort_values(by=["year", "quarter"])
    return data["company"], df


In [ ]:
def compute_kpis(df):
    df["revenue_yoy"] = df["revenue"].pct_change() * 100
    df["net_margin"] = df["net_income"] / df["revenue"] * 100
    df["debt_change"] = df["debt"].diff()
    return df


In [ ]:
def create_chunks(company, df):
    chunks = []
    for _, row in df.iterrows():
        text = f"""
        Company: {company}
        Period: {row['year']} {row['quarter']}
        Revenue: {row['revenue']}
        Net Income: {row['net_income']}
        Net Margin: {row['net_margin']:.2f}%
        Revenue YoY Change: {row['revenue_yoy']:.2f}%
        Cash Flow: {row['cash_flow']}
        Debt Level: {row['debt']}
        """
        chunks.append(text)
    return chunks


In [ ]:
def embed_chunks(chunks):
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode(chunks)
    return embeddings


In [ ]:
def store_vectors(chunks, embeddings):
    client = chromadb.Client(
        Settings(persist_directory="vector_store/chroma")
    )
    collection = client.get_or_create_collection("mdna_financials")

    for i, chunk in enumerate(chunks):
        collection.add(
            documents=[chunk],
            embeddings=[embeddings[i].tolist()],
            ids=[f"chunk_{i}"]
        )

    client.persist()
    return collection


In [ ]:
def retrieve_context(query, collection):
    results = collection.query(
        query_texts=[query],
        n_results=3
    )
    return results["documents"][0]


In [ ]:
def generate_mdna(context_chunks):
    openai.api_key = "YOUR_API_KEY"

    context = "\n".join(context_chunks)

    prompt = f"""
    You are a financial analyst.
    Using the financial data below, generate a professional MD&A report
    covering:
    - Results of Operations
    - Liquidity and Capital Resources
    - Risks and Outlook

    Financial Data:
    {context}

    Write in formal SEC MD&A style.
    """

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )

    return response.choices[0].message.content


In [ ]:
def main():
    company, df = load_financials("data/financials.json")
    df = compute_kpis(df)

    chunks = create_chunks(company, df)
    embeddings = embed_chunks(chunks)
    collection = store_vectors(chunks, embeddings)

    query = "Explain revenue growth, profitability, liquidity, and risks"
    context = retrieve_context(query, collection)

    mdna_report = generate_mdna(context)
    print("\n📄 GENERATED MD&A REPORT:\n")
    print(mdna_report)

if __name__ == "__main__":
    main()
